In [1]:
import pathlib

from fytok.modules.Equilibrium import Equilibrium
from fytok.utils.logger import logger
from spdm.view.View import display
WORKSPACE = "/home/salmon/workspace"  # "/ssd01/salmon_work/workspace/"
output_path = pathlib.Path(f"{WORKSPACE}/output/")

2023-12-06 21:29:13,100 [    spdm]     INFO: 
###################################################################################################

    ______      _____     _
   / ____/_  __|_   _|__ | | __
  / /_  / / / /  | |/ _ \| |/ /
 / __/ / /_/ /   | | (_) |   <
/_/    \__, /    |_|\___/|_|\_\
      /____/

 Copyright (c) 2021-present Zhi YU (Institute of Plasma Physics Chinese Academy of Sciences)
            
 url: https://gitee.com/openfusion/fytok_tutorial 
      https://github.com/fusion-yun/fytok_tutorial

 version = alpha  (spdm=beta extension=0.2.2.dev19+g14adade.d20231205, imas_version=v3_38_1_dirty)

 Run by salmon at 2023-12-06T21:29:12.833799.
 Job ID: fytok_salmon_surface_29643

###################################################################################################



In [2]:
# eq0 = Equilibrium(f"file+GEQdsk://{WORKSPACE}/gacode/neo/tools/input/profile_data/g141459.03890#equilibrium")
eq0 = Equilibrium("file+geqdsk:///home/salmon/workspace/fytok_tutorial/tutorial/data/g900003.00230_ITER_15MA_eqdsk16HR.txt#equilibrium",
                  code={"name":"freegs"})

2023-12-06 21:29:13,531 [    spdm]     INFO: Load module fytok.plugins.equilibrium.freegs
2023-12-06 21:29:13,535 [    spdm]     INFO: Load module spdm.plugins.data.plugin_geqdsk
2023-12-06 21:29:13,645 [    spdm]     INFO: Initialize module fytok.plugins.equilibrium.freegs 


In [3]:
profiles_2d=eq0.time_slice.current.profiles_2d

In [4]:
profiles_2d.grid

2023-12-06 21:29:13,676 [    spdm]     INFO: Load module spdm.mesh.mesh_rectangular


In [5]:
eq0.refresh()

2023-12-06 21:29:13,695 [    spdm]     INFO: Execute module fytok.plugins.equilibrium.freegs


In [6]:
profiles_2d.grid

RuntimeError: <class 'fytok.plugins.equilibrium.fy_eq.FyEquilibriumProfiles2D'> id=grid: 'getter' failed!